# Philippine Standard Geographic Code Reference File (PSGC)
##### As of June 30, 2020 
##### https://psa.gov.ph/classification/psgc/downloads/PSGC%202Q%202020%20Publication.xlsx

Create separate files for each administrative level

*Output CSV:*
PSGC_clean.csv - contains all data associating a BGY to a city, province, region code and name

*Other CSV per administrative region:*
- PSGC_2020_Reg.csv
- PSGC_2020_Prov.csv
- PSGC_2020_City.csv
- PSGC_2020_Mun.csv
- PSGC_2020_Dist.csv
- PSGC_2020_SubMun.csv
- PSGC_2020_Bgy.csv

In [1]:
# https://psa.gov.ph/classification/psgc/

In [2]:
import pandas as pd
import re

psgc_excel = pd.read_excel("PSGC 2Q 2020 Publication.xlsx", sheet_name="PSGC", usecols=range(0,7))
psgc_excel.head()

,Code,Name,Geographic Level,City Class,Income\nClassification,Urban / Rural\n(based on 2015 POPCEN),POPULATION\n(2015 POPCEN)
0,10000000,REGION I (ILOCOS REGION),Reg,NaN,NaN,NaN,5026128
1,12800000,ILOCOS NORTE,Prov,NaN,1st,NaN,593081
2,12801000,ADAMS,Mun,NaN,5th,NaN,1792
3,12801001,Adams (Pob.),Bgy,NaN,NaN,R,1792
4,12802000,BACARRA,Mun,NaN,3rd,NaN,32215


In [3]:
df = psgc_excel.copy()

In [4]:
new_cols = ['code','name','geographic_level','city_class','income_classification','urban_rural_2015_POPCEN', 'population_2015_POPCEN']
d = dict(zip(df.columns, new_cols))
df.rename(columns = d, inplace = True)

# Dataframe dimensions (row, column)
display(df.shape)

# Check if there are NaN in every column
nan_count = df.isnull().sum(axis = 0)
display(nan_count)
df.fillna('', inplace=True)

df['code'] = df.code.astype(str)

df.info()

(43798, 7)

code                           0
name                           0
geographic_level               2
city_class                 43652
income_classification      42083
urban_rural_2015_POPCEN     1761
population_2015_POPCEN         2
dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43798 entries, 0 to 43797
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   code                     43798 non-null  object
 1   name                     43798 non-null  object
 2   geographic_level         43798 non-null  object
 3   city_class               43798 non-null  object
 4   income_classification    43798 non-null  object
 5   urban_rural_2015_POPCEN  43798 non-null  object
 6   population_2015_POPCEN   43798 non-null  object
dtypes: object(7)
memory usage: 2.3+ MB


In [5]:
display( df[df['geographic_level'] == ''] )
# 32150	99700000	CITY OF ISABELA (Not a Province)	NaN				NaN
# 36747	129800000	COTABATO CITY (Not a Province)	NaN				NaN

df = df[df['geographic_level'] != '' ] # remove empty geographic_level

,code,name,geographic_level,city_class,income_classification,urban_rural_2015_POPCEN,population_2015_POPCEN
32150,99700000,CITY OF ISABELA (Not a Province),,,,,
36747,129800000,COTABATO CITY (Not a Province),,,,,


In [6]:
df['geographic_level'].nunique() # returns 7
geographic_level = list(df['geographic_level'].unique())
geographic_level

# ['Reg', 'Prov', 'Mun', 'Bgy', 'City', 'Dist', 'SubMun']

['Reg', 'Prov', 'Mun', 'Bgy', 'City', 'Dist', 'SubMun']

### REGIONS
 - Region
 - Province
 - City / Municipality
 - Bgy
 
### NCR
 - Region
 - District
 - City
 - SubMunicipality

In [7]:
psgc_db = []

reg_code = prov_code = mun_code = bgy_code = city_code = dist_code = submun_code = ''
reg_name = prov_name = mun_name = bgy_name = city_name = dist_name = submun_name = ''

for index, row in df.iterrows():
    row['code'] = str(row['code'])
    if row.geographic_level == 'Reg':
        reg_code, reg_name = row['code'], row['name']
        prov_code = mun_code = bgy_code = city_code = dist_code = submun_code = ''
        prov_name = mun_name = bgy_name = city_name = dist_name = submun_name = ''
    elif row.geographic_level == 'Prov':
        prov_code, prov_name  = row['code'], row['name']
        mun_code = bgy_code = city_code = dist_code = submun_code = ''
        mun_name = bgy_name = city_name = dist_name = submun_name = ''
    elif row.geographic_level == 'Dist': # For NCR
        dist_code, dist_name  = row['code'], row['name']
        prov_code = mun_code = bgy_code = city_code = submun_code = ''
        prov_name = mun_name = bgy_name = city_name = submun_name = ''
    elif row.geographic_level == 'Mun':
        mun_code, mun_name  = row['code'], row['name']
        bgy_code = city_code = dist_code = submun_code = ''
        bgy_name = city_name = dist_name = submun_name = ''
    elif row.geographic_level == 'City':
        city_code, city_name  = row['code'], row['name']
        mun_code = bgy_code = dist_code = submun_code = ''
        mun_name = bgy_name = dist_name = submun_name = ''
    elif row.geographic_level == 'SubMun':
        submun_code, submun_name  = row['code'], row['name']
        bgy_code = bgy_name = ''
    elif row.geographic_level == 'Bgy':
        bgy_code, bgy_name  = row['code'], row['name']
    else:
        continue
    
    r = list(row)
    r.extend( [reg_code, prov_code, mun_code, bgy_code, city_code, dist_code, submun_code] )
    r.extend( [reg_name, prov_name, mun_name, bgy_name, city_name, dist_name, submun_name] )
    psgc_db.append(r)

In [8]:
cols = list(df.columns)
cols.extend( [item + '_code' for item in geographic_level] )
cols.extend( [item + '_name' for item in geographic_level] )
new_df = pd.DataFrame(psgc_db, columns = cols)
new_df.to_csv('PSGC_clean.csv') ### 

In [9]:
new_df['geographic_level'].value_counts()

# Bgy       42046
# Mun        1488
# City        146
# Prov         81
# Reg          17
# SubMun       14
# Dist          4

Bgy       42046
Mun        1488
City        146
Prov         81
Reg          17
SubMun       14
Dist          4
Name: geographic_level, dtype: int64

In [10]:
for level in geographic_level:
    gl_list = new_df.loc[new_df.geographic_level == level].reset_index(drop=True)
    gl_list.to_csv('PSGC_2020_' + level + '.csv')
    print(level, len(gl_list))

Reg 17
Prov 81
Mun 1488
Bgy 42046
City 146
Dist 4
SubMun 14


In [11]:
new_df.shape

(43796, 21)

In [13]:
import pandas as pd

new_df = pd.read_csv('PSGC_clean.csv', na_filter=False)
new_df.tail()

,Unnamed: 0,code,name,geographic_level,city_class,income_classification,urban_rural_2015_POPCEN,population_2015_POPCEN,Reg_code,Prov_code,...,City_code,Dist_code,SubMun_code,Reg_name,Prov_name,Mun_name,Bgy_name,City_name,Dist_name,SubMun_name
43791,43791,168507005,Diaz (Romualdez),Bgy,,,R,611,160000000,168500000,...,,,,REGION XIII (Caraga),DINAGAT ISLANDS,TUBAJON,Diaz (Romualdez),,,
43792,43792,168507006,Roxas,Bgy,,,R,392,160000000,168500000,...,,,,REGION XIII (Caraga),DINAGAT ISLANDS,TUBAJON,Roxas,,,
43793,43793,168507007,San Roque (Pob.),Bgy,,,R,444,160000000,168500000,...,,,,REGION XIII (Caraga),DINAGAT ISLANDS,TUBAJON,San Roque (Pob.),,,
43794,43794,168507008,San Vicente (Pob.),Bgy,,,R,835,160000000,168500000,...,,,,REGION XIII (Caraga),DINAGAT ISLANDS,TUBAJON,San Vicente (Pob.),,,
43795,43795,168507009,Santa Cruz (Pob.),Bgy,,,R,1645,160000000,168500000,...,,,,REGION XIII (Caraga),DINAGAT ISLANDS,TUBAJON,Santa Cruz (Pob.),,,


In [24]:
# ### Generate SQL Statements
# sql = ''

# # Create Region Table
# sql = sql + 
# '''
# CREATE TABLE region (
#     'id' int AUTO_INCREMENT PRIMARY KEY,
#     'code' VARCHAR(8),
#     'name' VARCHAR(255),
#     'income_classification',
#     'urban_rural_2015_POPCEN',
#     'population_2015_POPCEN'
# );
# '''

# # Create Municipality
# sql = sql + 
# '''
# CREATE TABLE region (
#     'id' int AUTO_INCREMENT PRIMARY KEY,
#     'code' VARCHAR(8),
#     'name' VARCHAR(255),
#     'income_classification',
#     'urban_rural_2015_POPCEN',
#     'population_2015_POPCEN'
# );
# '''


In [16]:
dist = pd.read_csv('PSGC_2020_Dist.csv', na_filter=False)
dist.head()

,Unnamed: 0,code,name,geographic_level,city_class,income_classification,urban_rural_2015_POPCEN,population_2015_POPCEN,Reg_code,Prov_code,...,City_code,Dist_code,SubMun_code,Reg_name,Prov_name,Mun_name,Bgy_name,City_name,Dist_name,SubMun_name
0,0,133900000,"NCR, CITY OF MANILA, FIRST DISTRICT (Not a Pro...",Dist,,,,1780148,130000000,,...,,133900000,,NATIONAL CAPITAL REGION (NCR),,,,,"NCR, CITY OF MANILA, FIRST DISTRICT (Not a Pro...",
1,1,137400000,"NCR, SECOND DISTRICT (Not a Province)",Dist,,,,4650613,130000000,,...,,137400000,,NATIONAL CAPITAL REGION (NCR),,,,,"NCR, SECOND DISTRICT (Not a Province)",
2,2,137500000,"NCR, THIRD DISTRICT (Not a Province)",Dist,,,,2819388,130000000,,...,,137500000,,NATIONAL CAPITAL REGION (NCR),,,,,"NCR, THIRD DISTRICT (Not a Province)",
3,3,137600000,"NCR, FOURTH DISTRICT (Not a Province)",Dist,,,,3627104,130000000,,...,,137600000,,NATIONAL CAPITAL REGION (NCR),,,,,"NCR, FOURTH DISTRICT (Not a Province)",


In [17]:
prov = pd.read_csv("PSGC_2020_Prov.csv", na_filter=False)
display( prov.shape )
prov.head(3)

(81, 22)

,Unnamed: 0,code,name,geographic_level,city_class,income_classification,urban_rural_2015_POPCEN,population_2015_POPCEN,Reg_code,Prov_code,...,City_code,Dist_code,SubMun_code,Reg_name,Prov_name,Mun_name,Bgy_name,City_name,Dist_name,SubMun_name
0,0,12800000,ILOCOS NORTE,Prov,,1st,,593081,10000000,12800000,...,,,,REGION I (ILOCOS REGION),ILOCOS NORTE,,,,,
1,1,12900000,ILOCOS SUR,Prov,,1st,,689668,10000000,12900000,...,,,,REGION I (ILOCOS REGION),ILOCOS SUR,,,,,
2,2,13300000,LA UNION,Prov,,1st,,786653,10000000,13300000,...,,,,REGION I (ILOCOS REGION),LA UNION,,,,,


In [20]:
import pandas as pd
bgy = pd.read_csv("PSGC_2020_Bgy.csv", na_filter=False)
print(bgy.shape)
display(bgy.tail(3))

(42046, 22)


/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,code,name,geographic_level,city_class,income_classification,urban_rural_2015_POPCEN,population_2015_POPCEN,Reg_code,Prov_code,...,City_code,Dist_code,SubMun_code,Reg_name,Prov_name,Mun_name,Bgy_name,City_name,Dist_name,SubMun_name
42043,42043,168507007,San Roque (Pob.),Bgy,,,R,444,160000000,168500000,...,,,,REGION XIII (Caraga),DINAGAT ISLANDS,TUBAJON,San Roque (Pob.),,,
42044,42044,168507008,San Vicente (Pob.),Bgy,,,R,835,160000000,168500000,...,,,,REGION XIII (Caraga),DINAGAT ISLANDS,TUBAJON,San Vicente (Pob.),,,
42045,42045,168507009,Santa Cruz (Pob.),Bgy,,,R,1645,160000000,168500000,...,,,,REGION XIII (Caraga),DINAGAT ISLANDS,TUBAJON,Santa Cruz (Pob.),,,


In [27]:
bgy['Reg_name'].unique()

array(['REGION I (ILOCOS REGION)', 'REGION II (CAGAYAN VALLEY)',
       'REGION III (CENTRAL LUZON)', 'REGION IV-A (CALABARZON)',
       'MIMAROPA REGION', 'REGION V (BICOL REGION)',
       'REGION VI (WESTERN VISAYAS)', 'REGION VII (CENTRAL VISAYAS)',
       'REGION VIII (EASTERN VISAYAS)', 'REGION IX (ZAMBOANGA PENINSULA)',
       'REGION X (NORTHERN MINDANAO)', 'REGION XI (DAVAO REGION)',
       'REGION XII (SOCCSKSARGEN)', 'NATIONAL CAPITAL REGION (NCR)',
       'CORDILLERA ADMINISTRATIVE REGION (CAR)',
       'AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM)',
       'REGION XIII (Caraga)'], dtype=object)

### Prepared by: RUTH ANN CABRIA
Contact: racabria@gmail.com